In [10]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import torch
import json

In [11]:
from typing import List
from ultralytics.engine.results import Results
import os


FRAME_ANALYSIS_FREQ = 4
VIDEO_DIR = "vid"
USED_CLASSES = [2, 5, 7] # 2 - car, 5 - bus, 7 - truck

init_keys = ["file_name", "car", "quantity_car", "average_speed_car", "van", "quantity_van", "average_speed_van", "bus", "quantity_bus", "average_speed_bus"]
result_df = pd.DataFrame(dict.fromkeys(init_keys, []))
print(result_df)

vids = os.listdir(VIDEO_DIR)
print("Доступные видео: ", vids)
for vid_fname_idx in range(len(vids)):
    vid_fname = vids[vid_fname_idx]

    if "placeholder" in vid_fname:
        continue

    model = YOLO('yolov8x.pt')

    print("Сейчас обрабатывается", vid_fname)
    vid_path = f"{VIDEO_DIR}/{vid_fname}"
    vid_capture = cv2.VideoCapture(vid_path)

    appeared_ids = []
    count_dict = dict.fromkeys(USED_CLASSES, 0)

    frame_idx = 0

    while vid_capture.isOpened():
        success, frame = vid_capture.read()

        if success:
            if frame_idx % FRAME_ANALYSIS_FREQ == 0:
                # !!! Изменить verbose на True если нужна инфа от модели !!!
                track_results : List[Results] = model.track(frame, persist=True, classes=USED_CLASSES, verbose=False) 
                annotated_frame = track_results[0].plot()

                bb_on_frame_ids : np.ndarray = track_results[0].boxes.id.numpy()
                bb_on_frame_cls : np.ndarray = track_results[0].boxes.cls.numpy()

                for idx in range(len(bb_on_frame_ids)):
                    if (bb_on_frame_ids[idx] not in appeared_ids):
                        appeared_ids.append(bb_on_frame_ids[idx])
                        count_dict[bb_on_frame_cls[idx]] += 1

                # print(f"Количество обьектов класса на {frame_idx} фрейм", count_dict)

                annotated_frame = cv2.resize(annotated_frame, (1280, 640))                
                cv2.imshow("YOLOv8 Tracking", annotated_frame)
                key = cv2.waitKey(2)
    
                if  key == 27:
                    break
            frame_idx += 1
        else:
            break
    
    print(f"Финальное количество объектов в видео {vid_fname}", count_dict)
    vid_fname_formatless = vid_fname.split(".")[0]
    # !!! Там где нолики места для средней скорости !!!
    data_inserted = [vid_fname_formatless, "car", count_dict[2], 0, "van", count_dict[7], 0, "bus", count_dict[5], 0] 

    result_df.loc[vid_fname_idx] = data_inserted

    vid_capture.release()
    cv2.destroyAllWindows()

print(result_df)

Empty DataFrame
Columns: [file_name, car, quantity_car, average_speed_car, van, quantity_van, average_speed_van, bus, quantity_bus, average_speed_bus]
Index: []
Доступные видео:  ['KRA-2-7-2023-08-23-evening.mp4', 'KRA-25-73-2023-08-24-morning.mp4', 'placeholder']


100%|██████████| 131M/131M [00:11<00:00, 11.5MB/s] 


Сейчас обрабатывается KRA-2-7-2023-08-23-evening.mp4
requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ---------------------------------------- 1.5/1.5 MB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 2.8/2.8 MB 9.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 4.9s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Финальное количество объектов в видео KRA-2-7-2023-08-23-evening.mp4 {2: 6, 5: 2, 7: 0}
Сейчас обрабатывается KRA-25-73-2023-08-24-morning.mp4
Финальное количество объектов в видео KRA-25-73-2023-08-24-morning.mp4 {2: 4, 5: 1, 7: 1}
                      file_name  car  quantity_car  average_speed_car  van  \
0    KRA-2-7-2023-08-23-evening  car             6                  0  van   
1  KRA-25-73-2023-08-24-morning  car             4                  0  van   

   quantity_van  average_speed_van  bus  quantity_bus  average_speed_bus  
0